<a href="https://colab.research.google.com/github/hyuna0926/cp2_phase2/blob/main/recommend/class_recommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 필요 라이브러리 로드

In [1]:
! pip install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.6/18.6 MB 18.2 MB/s eta 0:00:00


In [1]:
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix
from tqdm.notebook import tqdm
import numpy as np
from datetime import datetime

import warnings
# 경고 제거
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from implicit.als import AlternatingLeastSquares as ALS
import implicit
import random
import os

In [2]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [3]:
path = '/content/drive/MyDrive/CP2_Phase2/'
'''
cart_purchase: 구매이력
product : 상품 정보(31121)
customer_info : 회원 정보(0~100,000)
'''
cart_purchase = pd.read_parquet(path + 'cart_purchase.parquet')
product = pd.read_parquet(path + 'product.parquet')
customer_info = pd.read_parquet(path + 'customer_info.parquet')

# 데이터 만들기

In [4]:
class Data():
  '''
  c_idx를 입력했을 때 원하는 데이터 나올 수 있게 하기?~!
  1. 아예 없는 경우
  2. 구매하지 않은 회원(customer_info.parquet)
  3. 20개 미만 산 회원(cart_purchase)
  4. 20개 이상 산 회원(cart_purchase)
  '''
  def __init__(self):
    pass


  def customer_info(self, c_idx):
    # 비회원
    if c_idx not in customer_info['c_idx']:
      print(c_idx, '님은 회원정보가 없습니다.')

    # 구매하지 않는 회원
    elif c_idx not in cart_purchase['c_idx']:
      print(c_idx, '님은 구매이력이 없습니다.')

    # 20개 이상 산 회원
    elif c_idx in self.customer_data()[2]['c_idx']: # upper
      print(c_idx, '님은 20개 이상 구매한 회원입니다. ALS 진행')
      
    else:
      print(c_idx, '님은 20개 미만 구매한 회원입니다. CB 진행')

    return c_idx

  
  def customer_data(self):
    '''
    ALS와 CB 데이터 만들기

    return 
    train, test : ALS
    upper, lower : CB(20개 기준으로 나눔)
    '''
    df = cart_purchase.copy()
    df['values'] = 1  # 구매, 장바구니니까 1로 implicit 데이터

    df_group = df.groupby(['c_idx'], as_index=False).count()
    
    # ALS(20개 이상)
    upper = df[df['c_idx'].isin(df_group.query('values>=20').c_idx)] # 20개 이상
    test = upper.groupby('c_idx').sample(frac=0.2, random_state=42) # als_test
    train = upper.drop(test.index) # als_train
    
    # CB(20개 미만)
    lower = df[df['c_idx'].isin(df_group.query('values<20').c_idx)] # 20개 미만

    return train, test, upper, lower
  


  def no_purchase_data(self):
    '''
    회원정보는 있지만 구매하지 않은 사람을 위한 나이별 데이터 만들기
    customer_info와 cart_purchase 합치기

    return
    youth : 20대 이하 구매 이력
    age_20 : 20대 구매 이력
    age_30 : 30대 이상 구매 이력
    '''
    customer_c = customer_info.copy()
    customer_c.birthdate = customer_c.birthdate.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
    customer_c['age']=customer_c.birthdate.apply(lambda x: datetime.today().year - x.year)

    cus_pur = pd.merge(cart_purchase, customer_c, on=['c_idx','customer_id'], how='left')

    #데이터 분리
    youth = cus_pur.query("age<20")  # 20대 이하 구매 이력
    age_20 = cus_pur.query('age>=20 and age<30')  # 20대 구매 이력
    age_30 = cus_pur.query('age>=30')  # 30대 이상 구매 이력

    return youth, age_20, age_30




In [90]:
data = Data()
data.customer_info(7893)

7893 님은 20개 이상 구매한 회원입니다. ALS 진행


7893

In [91]:
train, test, upper, lower = data.customer_data() # 데이터 잘 나눠짐

In [92]:
print(train.shape, test.shape)

(1297193, 7) (334749, 7)


In [93]:
youth, age_20, age_30 = data.no_purchase_data() # 데이터 잘 나눠짐

# CB

In [12]:
class CB_recommend():
  '''
  20개 미만 구매한 고객에게 성별별로 추천하기
  tf-idf를 이용한 컨텐츠 기반 추천시스템
  '''
  def __init__(self):
    pass
  
  def CB_data(self):
    '''
    CB data 만들어주기
    성별 나눠줌
    - 남자 ['Men','Boys','Unisex'] 17712개
    - 여자 ['Women','Girls','Unisex'] 15157개
    - Unisex는 모두 포함

    return 
    Men : 남성 상품 정보
    Women : 여성 상품 정보
    product_c : 중복값을 제거한 전체 상품 정보
    '''

    # product 중복값 제거
    product_c = product.drop_duplicates(subset=['productDisplayName'], keep='first',ignore_index=True)
    # 결측값 제거
    df = product_c.copy()
    df.dropna(subset=['productDisplayName','year'],inplace=True)
    col = ['baseColour','season','usage']
    df[col]=df[col].astype('object')
    df[col] = df[col].fillna("unknown") #결측값 채워주기
    df[col]=df[col].astype('category')
    
    #tf-idf를 위해 컬럼 만들어주기
    df['features'] = df[['gender','articleType','baseColour','season','usage']].apply(' '.join, axis=1)

    # 성별 나눠주기
    men = ['Men','Boys','Unisex']
    women = ['Women','Girls','Unisex']
    Men = df[df['gender'].isin(men)]
    Women = df[df['gender'].isin(women)]

    return Men, Women ,product_c

  def tfidf(self):
    '''
    tf-idf을 이용한 컨텐츠 기반 추천시스템
    남녀별로 코사인 유사도 진행 
    
    return cosine_men, cosine_women
    '''
    Men, Women, product_c = self.CB_data()

    tfidf_m = TfidfVectorizer()
    tfidf_men = tfidf_m.fit_transform(Men['features'])
    tfidf_w = TfidfVectorizer()
    tfidf_women = tfidf_w.fit_transform(Women['features'])


    #코사인 유사도
    cosine_men = pd.DataFrame(cosine_similarity(tfidf_men,tfidf_men),index = Men.p_idx, columns=Men.p_idx)
    cosine_women = pd.DataFrame(cosine_similarity(tfidf_women,tfidf_women),index = Women.p_idx, columns=Women.p_idx)

    return cosine_men, cosine_women


  def recommend(self, c_idx, k=25):
    '''
    Data클래스에 있는 데이터 들고오기
    남자/여자 나눠서 추천
    최근에 산 제품과 유사한 상품 k개 추천

    return recommend(유사한 상품 k개)
    '''
    data = Data()
    lower = data.customer_data()[3] # 20개 이하 구매한 사람

    Men, Women, product_c = self.CB_data()
    cosine_men, cosine_women = self.tfidf()

    # 고객이 최근에 산 상품
    buy_recent = lower[lower['c_idx']==c_idx].sort_values(by='created_at', ascending=False)[:1]  
    p_idx = buy_recent.p_idx.values[0]


    if p_idx in Men['p_idx']: #상품 p_idx가 Men에 들어가있으면
      men_sim = cosine_men[p_idx].sort_values(ascending=False).index
      recommend = product_c[product_c['p_idx'].isin(men_sim)][:k]
    
    else: # Women에 들어가있으면
      women_sim = cosine_women[p_idx].sort_values(ascending=False).index
      recommend = product_c[product_c['p_idx'].isin(women_sim)][:k]

    return recommend
    
   

In [15]:
cb_recommend = CB_recommend()
cb_recommend.recommend(6934)

,product_id,gender,masterCategory,subCategory,articleType,baseColour,season,year,usage,productDisplayName,p_idx
0,15970,Men,Apparel,Topwear,Shirts,Navy Blue,Fall,2011,Casual,Turtle Check Men Navy Blue Shirt,27781
1,39386,Men,Apparel,Bottomwear,Jeans,Blue,Summer,2012,Casual,Peter England Men Party Blue Jeans,20698
3,21379,Men,Apparel,Bottomwear,Track Pants,Black,Fall,2011,Casual,Manchester United Men Solid Black Track Pants,16433
4,53759,Men,Apparel,Topwear,Tshirts,Grey,Summer,2012,Casual,Puma Men Grey T-shirt,22200
5,1855,Men,Apparel,Topwear,Tshirts,Grey,Summer,2011,Casual,Inkfruit Mens Chain Reaction T-shirt,12874
6,30805,Men,Apparel,Topwear,Shirts,Green,Summer,2012,Ethnic,Fabindia Men Striped Green Shirt,8473
8,29114,Men,Accessories,Socks,Socks,Navy Blue,Summer,2012,Casual,Puma Men Pack of 3 Socks,22387
9,30039,Men,Accessories,Watches,Watches,Black,Winter,2016,Casual,Skagen Men Black Watch,26021
10,9204,Men,Footwear,Shoes,Casual Shoes,Black,Summer,2011,Casual,Puma Men Future Cat Remix SF Black Casual Shoes,22136
12,18653,Men,Footwear,Flip Flops,Flip Flops,Black,Fall,2011,Casual,Fila Men Cush Flex Black Slippers,9695


# Nobuy_recommend